In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import t, shapiro
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels
from all_functions import refactorGiniWorldBankDf


In [2]:
revenuMondial = pd.read_csv("../data/data-projet7.csv", parse_dates=True)
display(revenuMondial.head())
display(revenuMondial.shape)

,country,year_survey,quantile,nb_quantiles,income,gdpppp
0,ALB,2008,1,100,"728,89795",7297
1,ALB,2008,2,100,"916,66235",7297
2,ALB,2008,3,100,"1010,916",7297
3,ALB,2008,4,100,"1086,9078",7297
4,ALB,2008,5,100,"1132,6997",7297


(11599, 6)

In [3]:
print(len(revenuMondial) - len(revenuMondial.drop_duplicates()), 'doublon') # Aucun doublon

0 doublon


In [4]:
# Ci-dessus, les revenues mondiale pour une période donnée

In [5]:
# Premère étape. Identifiquation des pays

In [6]:
countryCodes = pd.read_csv("../data/iso.csv", parse_dates=True)
countryCodes.head(5)

,name,iso
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,Andorra,AND
4,Angola,AGO


In [7]:
merged_df = pd.merge(revenuMondial, countryCodes, left_on='country', right_on='iso', how='left')
# On veux obtenir le nom des pays via leurs code iso.
incomes = merged_df[['name','country', 'year_survey', 'quantile','nb_quantiles','income','gdpppp']]
incomes = incomes.rename(columns={"name":"nation"})
incomes = incomes.rename(columns={"year_survey":"year"})
# On transforme en float 
incomes.replace(',', '.', regex=True, inplace=True)
incomes['income'] = incomes['income'].astype(float)
incomes['gdpppp'] = incomes['gdpppp'].astype(float)
incomes['year'] = incomes['year'].astype(int)
incomes.head(5)

,nation,country,year,quantile,nb_quantiles,income,gdpppp
0,Albania,ALB,2008,1,100,728.89795,7297.0
1,Albania,ALB,2008,2,100,916.66235,7297.0
2,Albania,ALB,2008,3,100,1010.91600,7297.0
3,Albania,ALB,2008,4,100,1086.90780,7297.0
4,Albania,ALB,2008,5,100,1132.69970,7297.0


In [8]:
# Information dataFrame

In [9]:
incomes["year"] = incomes["year"].astype(int)
# Résumé des dates minimales et maximales
date_min = incomes['year'].min()
date_max = incomes['year'].max()
# Résumé des revenus minimales et maximales
incomeMin = incomes['income'].min()
incomeMax = incomes['income'].max()
# Affichage des résultats
print(f"Date minimale : {date_min}")
print(f"Date maximale : {date_max}")

Date minimale : 2004
Date maximale : 2011


In [10]:
firstQuantile = incomes[incomes['quantile'] == 1]
lowestFistQuantile = firstQuantile.sort_values(by='income')
hundredQuantile = incomes[incomes['quantile'] == 100]
highestFistQuantile = hundredQuantile.sort_values(by='income', ascending=False)
highestFistQuantile.iloc[0]

nation          United States
country                   USA
year                     2008
quantile                  100
nb_quantiles              100
income              176928.55
gdpppp                43261.0
Name: 11098, dtype: object

In [11]:
arrIncomesYears = incomes['year'].sort_values(ascending=True).unique()
arrIncomesYears = [str(value) for value in arrIncomesYears]
arrIncomesYears

['2004', '2006', '2007', '2008', '2009', '2010', '2011']

In [12]:
nombrePays = incomes['country'].nunique()
print(f"Nombre de pays par année: {nombrePays}")

Nombre de pays par année: 116


Traitement des donnée manquante

In [13]:
# Grouper par pays
groupedByCountry = incomes.groupby('country')
# Liste des quantiles attendus (1 à 100)
expectedQuantiles = set(range(1, 101))
# Créer une liste pour stocker les pays avec des quantiles manquants
countriesWithMissingQuantiles = []

# Parcourir chaque groupe (pays) et vérifier les quantiles manquants
for country, group in groupedByCountry:
    actualQuantiles = set(group['quantile'])
    missingQuantiles = expectedQuantiles - actualQuantiles
    
    if missingQuantiles:
        countriesWithMissingQuantiles.append((country, missingQuantiles))

# Afficher les résultats
if countriesWithMissingQuantiles:
    print("Pays avec des quantiles manquants :")
    for country, missingQuantiles in countriesWithMissingQuantiles:
        print(f"{country}: Quantiles manquants {missingQuantiles}")
else:
    print("Aucun pays avec des quantiles manquants.")

Pays avec des quantiles manquants :
LTU: Quantiles manquants {41}


In [14]:
ltu = 'LTU'
moyenneLTUIncome = incomes.loc[(incomes['country'] == ltu) & (incomes['quantile'] == 40) 
            | (incomes['country'] == ltu) & (incomes['quantile'] == 42)]['income'].mean();
moyenneLTUIncome

4882.14065

In [15]:
dfMissingRowLTU = pd.DataFrame({'nation':'Lithuania','country': [ltu], 'year': [2008], 'quantile': [41], 
                           'nb_quantiles': [100], 'income': [moyenneLTUIncome], 'gdpppp': 17571.0});

dfMissingRowLTU

,nation,country,year,quantile,nb_quantiles,income,gdpppp
0,Lithuania,LTU,2008,41,100,4882.14065,17571.0


In [16]:
incomes = pd.concat([incomes, dfMissingRowLTU], ignore_index=True, sort=True)
display(incomes.shape)
incomes.head()

(11700, 7)

,country,gdpppp,income,nation,nb_quantiles,quantile,year
0,ALB,7297.0,728.89795,Albania,100,1,2008
1,ALB,7297.0,916.66235,Albania,100,2,2008
2,ALB,7297.0,1010.91600,Albania,100,3,2008
3,ALB,7297.0,1086.90780,Albania,100,4,2008
4,ALB,7297.0,1132.69970,Albania,100,5,2008


In [17]:
incomes[incomes['country'] == 'LTU'].head(1)

,country,gdpppp,income,nation,nb_quantiles,quantile,year
6200,LTU,17571.0,566.3453,Lithuania,100,1,2008


In [18]:
countriesNullsdpppp = incomes.loc[incomes['gdpppp'].isnull(), 'country'].unique()
countriesNullsdpppp

array(['XKX', 'PSE'], dtype=object)

In [19]:
xkx = countriesNullsdpppp[0];
pse = countriesNullsdpppp[1];

Je récupére le dataset de la worlbank afin de récuperer les gd pppp 


In [20]:
gdpppp = pd.read_csv("../data/worldbank-gdpppp.csv", parse_dates=True, sep=';')
gdpppp = gdpppp.iloc[: , :-1]
copyGd = gdpppp.copy
gdpppp.head(1)

,Country Name,Country Code,Indicator Name,1960,1961,1962,1963,1964,1965,1966,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,GDP per capita (constant LCU),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,47461.615474,46386.672241,45804.246592,46317.795605,49316.830489,50242.212175,48863.866312,37095.214575,47369.425924,52368.693789


In [21]:
pseGd = gdpppp[gdpppp['Country Code'] == pse]
pseGdppp = pseGd['2009'].values[0]
pseGdppp

2840.01594969395

In [22]:
getXKXGd = gdpppp[gdpppp['Country Code'] == xkx]
xkxGdppp = getXKXGd['2009'].values[0]
xkxGdppp

2109.4339172761

In [23]:
incomes.loc[incomes['country'] == xkx, 'gdpppp'] = xkxGdppp
incomes.loc[incomes['country'] == xkx, 'nation'] = 'Kosovo'
incomes.loc[incomes['country'] == pse, 'gdpppp'] = pseGdppp
incomes.loc[incomes['country'] == pse, 'nation'] = 'Palestine'
incomes[incomes['country'] == xkx].head(1)

,country,gdpppp,income,nation,nb_quantiles,quantile,year
5800,XKX,2109.433917,437.8937,Kosovo,100,1,2008


In [24]:
incomes[incomes['country'] == pse].head(1)

,country,gdpppp,income,nation,nb_quantiles,quantile,year
11299,PSE,2840.01595,195.2899,Palestine,100,1,2009


In [25]:
incomes.head()

,country,gdpppp,income,nation,nb_quantiles,quantile,year
0,ALB,7297.0,728.89795,Albania,100,1,2008
1,ALB,7297.0,916.66235,Albania,100,2,2008
2,ALB,7297.0,1010.91600,Albania,100,3,2008
3,ALB,7297.0,1086.90780,Albania,100,4,2008
4,ALB,7297.0,1132.69970,Albania,100,5,2008


#Indice de gini 

In [26]:
giniPath = "../data/gini.csv"
# Nettoyage de base des données.
gini = pd.read_csv(giniPath, delimiter=',', parse_dates=True)
gini = gini.drop('Indicator Name', axis=1)
gini = gini.drop('Indicator Code', axis=1)
gini = gini.dropna(axis=1, how='all')
gini = gini.rename(columns={"Country Code":"country"})
gini = gini.rename(columns={"Country Name":"nation"})
gini = gini.dropna(how='all')
gini = gini.dropna(axis=1, how='all')
gini.head()

,nation,country,1979,1980,1981,1982,1983,1984,1985,1986,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,42.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.0,NaN,NaN,NaN,29.0,NaN,NaN,NaN,NaN,NaN
4,Andorra,AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Verification si nous avons des données d'indice de gini comparer au df gini
giniColumns = []
for value in gini.columns:
    giniColumns.append(value)
    
giniDates = [year for year in giniColumns if year in (['2004', '2006', '2007', '2008', '2009', '2010', '2011'])]
giniMelt = pd.melt(gini, id_vars =['nation', 'country'], value_vars = giniDates, var_name ='year', value_name ='gini')
giniMelt['gini'] = round(giniMelt['gini']/100, 2)
giniMelt.dropna(inplace=True)
giniMelt['year'] = giniMelt['year'].astype(int)
giniMelt.head()

,nation,country,year,gini
7,Argentina,ARG,2004,0.49
8,Armenia,ARM,2004,0.38
11,Australia,AUS,2004,0.33
12,Austria,AUT,2004,0.30
13,Azerbaijan,AZE,2004,0.27


In [28]:
# Merge 
incomes = pd.merge(incomes, giniMelt, how='left', on=['country', 'year'])
incomes.head()

,country,gdpppp,income,nation_x,nb_quantiles,quantile,year,nation_y,gini
0,ALB,7297.0,728.89795,Albania,100,1,2008,Albania,0.3
1,ALB,7297.0,916.66235,Albania,100,2,2008,Albania,0.3
2,ALB,7297.0,1010.91600,Albania,100,3,2008,Albania,0.3
3,ALB,7297.0,1086.90780,Albania,100,4,2008,Albania,0.3
4,ALB,7297.0,1132.69970,Albania,100,5,2008,Albania,0.3


In [29]:
incomes = incomes.drop(columns=['nation_y'])
incomes = incomes.rename(columns={"nation_x":"nation"})
incomes.head()

,country,gdpppp,income,nation,nb_quantiles,quantile,year,gini
0,ALB,7297.0,728.89795,Albania,100,1,2008,0.3
1,ALB,7297.0,916.66235,Albania,100,2,2008,0.3
2,ALB,7297.0,1010.91600,Albania,100,3,2008,0.3
3,ALB,7297.0,1086.90780,Albania,100,4,2008,0.3
4,ALB,7297.0,1132.69970,Albania,100,5,2008,0.3


In [30]:
giniMelt

,nation,country,year,gini
7,Argentina,ARG,2004,0.49
8,Armenia,ARM,2004,0.38
11,Australia,AUS,2004,0.33
12,Austria,AUT,2004,0.30
13,Azerbaijan,AZE,2004,0.27
...,...,...,...,...
1828,Tanzania,TZA,2011,0.38
1830,Ukraine,UKR,2011,0.25
1832,Uruguay,URY,2011,0.42
1843,Kosovo,XKX,2011,0.28


In [31]:
# Indice Manquant
missingGini = incomes[incomes['gini'].isna()][['country', 'year']]
missingCountry = list(missingGini['country'].unique())
len(missingCountry)

26

In [32]:
print(missingCountry)

['AZE', 'CAN', 'CHN', 'EGY', 'GHA', 'GTM', 'HRV', 'IND', 'IRN', 'IRQ', 'ISR', 'KEN', 'KHM', 'XKX', 'LKA', 'MAR', 'MKD', 'MLI', 'MYS', 'NGA', 'PAK', 'SRB', 'TWN', 'USA', 'YEM', 'COD']


In [33]:
# df a part pour isolé 
getMissedGini = incomes[incomes.isna().any(axis=1)].drop_duplicates(subset=['country', 'quantile', 'year'])
getMissedGini.head()

,country,gdpppp,income,nation,nb_quantiles,quantile,year,gini
400,AZE,8101.0,397.53983,Azerbaijan,100,1,2008,NaN
401,AZE,8101.0,597.77320,Azerbaijan,100,2,2008,NaN
402,AZE,8101.0,705.47003,Azerbaijan,100,3,2008,NaN
403,AZE,8101.0,785.52610,Azerbaijan,100,4,2008,NaN
404,AZE,8101.0,841.75244,Azerbaijan,100,5,2008,NaN


In [34]:
listGini = []
# Itérer à travers chaque code de pays
for countryCode in getMissedGini['country']:
    # Sélectionner les données de revenu pour le pays actuel
    revenuTemporaire = getMissedGini[getMissedGini['country'] == countryCode]['income'].values
    # Obtenir le nombre de points de données
    numPointsDonnees = len(revenuTemporaire)
    # Calculer la courbe de Lorenz
    courbeLorenz = np.cumsum(np.sort(revenuTemporaire)) / revenuTemporaire.sum()
    # Calculer l'aire sous la courbe de Lorenz (AUC)
    AUC = (courbeLorenz.sum() - courbeLorenz[-1] / 2 - courbeLorenz[0] / 2) / numPointsDonnees
    # Calculer le coefficient de Gini
    S = 0.5 - AUC
    coefficientGini = round(2 * S, 2)
    # Ajouter le coefficient de Gini calculé à la liste
    listGini.append(coefficientGini)

In [35]:
addMissedGini = getMissedGini.copy()
addMissedGini.loc[:,'gini'] = listGini
addMissedGini.head()

,country,gdpppp,income,nation,nb_quantiles,quantile,year,gini
400,AZE,8101.0,397.53983,Azerbaijan,100,1,2008,0.31
401,AZE,8101.0,597.77320,Azerbaijan,100,2,2008,0.31
402,AZE,8101.0,705.47003,Azerbaijan,100,3,2008,0.31
403,AZE,8101.0,785.52610,Azerbaijan,100,4,2008,0.31
404,AZE,8101.0,841.75244,Azerbaijan,100,5,2008,0.31


In [36]:
# Mise en place dans incomes 
for countryCode in list(addMissedGini['country'].unique()) :
    incomes.loc[incomes['country'] == countryCode,'gini'] = (addMissedGini.loc[addMissedGini['country'] == countryCode, 'gini'])


In [37]:
incomes.loc[incomes['country'] == 'MKD', 'gini'] = 0.31

In [38]:
incomes[incomes['gini'].isna()] 

,country,gdpppp,income,nation,nb_quantiles,quantile,year,gini


In [39]:
population = pd.read_csv("../data/population.csv", parse_dates=True)
population = population.dropna(axis=1, how='all')
population = population.rename(columns={"Country Code":"country"})
population = population[['country', '2004', '2006', '2007', '2008', '2009', '2010', '2011']]
population = pd.melt(population, id_vars=['country'], var_name='year', value_name='population')
population['population'].fillna(0, inplace=True)
population['population'] = (population['population'].astype(int) * 1000)

if 'INX' in population['country'].values:
    population = population[population['country'] != 'INX']
population

,country,year,population
0,ABW,2004,98737000
1,AFG,2004,24726684000
2,AGO,2004,18758145000
3,ALB,2004,3026939000
4,AND,2004,76244000
...,...,...,...
1843,XKX,2011,1791000000
1844,YEM,2011,23807588000
1845,ZAF,2011,52004172000
1846,ZMB,2011,14023193000


In [40]:
population[population['population'] == 0]

,country,year,population


In [41]:
incomes = pd.merge(incomes, population[['country', 'population']], on='country', how='left')
incomes['population'].fillna(0, inplace=True)
incomes['population'] = incomes['population'].astype(int)
incomes.head()

,country,gdpppp,income,nation,nb_quantiles,quantile,year,gini,population
0,ALB,7297.0,728.89795,Albania,100,1,2008,0.3,3026939000
1,ALB,7297.0,728.89795,Albania,100,1,2008,0.3,2992547000
2,ALB,7297.0,728.89795,Albania,100,1,2008,0.3,2970017000
3,ALB,7297.0,728.89795,Albania,100,1,2008,0.3,2947314000
4,ALB,7297.0,728.89795,Albania,100,1,2008,0.3,2927519000


In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
plt.bar(incomes['year'], incomes['population'])
plt.xlabel('Année')
plt.ylabel('Population')
plt.title("Représentation du df selon la population et les années")
plt.show()

In [43]:
years = incomes['year'].unique()
nbCountry = incomes['country'].nunique()
totalPop = incomes['population'].sum()
populationMondiale = 8_000_000_000  
popCouvert =  (totalPop / populationMondiale) 
reponse = "Ce sont des centiles permet de réduire la taille de l'échantillon, tout en préservant suffisamment d'informations pour pouvoir l'analyser"
reponse2 = "2007 est l'années la plus représentatives de l'échantillon."
reponse3 = "Le $PPP est une unité qui permet de comparer le pouvoir d'achat entre deux pays sans distorsion due aux taux de change."
print("Années des données utilisées de 2006 à 2011")
print("Les années ou nous retrouvons le plus de données est 2007")
print("Nombre de pays présents:", nbCountry)
print("Population mondiale couverte par l'analyse : {:.2f}%".format(popCouvert))
print(reponse)
print(reponse2)
print(reponse3)

Années des données utilisées de 2006 à 2011
Les années ou nous retrouvons le plus de données est 2007
Nombre de pays présents: 116
Population mondiale couverte par l'analyse : 540165.35%
Ce sont des centiles permet de réduire la taille de l'échantillon, tout en préservant suffisamment d'informations pour pouvoir l'analyser
2007 est l'années la plus représentatives de l'échantillon.
Le $PPP est une unité qui permet de comparer le pouvoir d'achat entre deux pays sans distorsion due aux taux de change.


# Mission 2 
Diversité des revenues par pays. Pour ce faire 

In [44]:
print(str(len(incomes['country'].unique())) + ' nb pays')

116 nb pays


In [45]:
incomes['meanIncome'] = incomes.groupby('country')['income'].transform('mean').round(2)
incomes.head()

,country,gdpppp,income,nation,nb_quantiles,quantile,year,gini,population,meanIncome
0,ALB,7297.0,728.89795,Albania,100,1,2008,0.3,3026939000,2994.83
1,ALB,7297.0,728.89795,Albania,100,1,2008,0.3,2992547000,2994.83
2,ALB,7297.0,728.89795,Albania,100,1,2008,0.3,2970017000,2994.83
3,ALB,7297.0,728.89795,Albania,100,1,2008,0.3,2947314000,2994.83
4,ALB,7297.0,728.89795,Albania,100,1,2008,0.3,2927519000,2994.83


In [46]:
# clustering 

In [47]:
selectCountries = ['France', 'United States', 'China', 'India', 'Brazil', 'South Africa', 'Germany', 'Australia', 'Canada', 'Japan']
incomesSelected = incomes[incomes['nation'].isin(selectCountries)]
incomesSelected

,country,gdpppp,income,nation,nb_quantiles,quantile,year,gini,population,meanIncome
8400,BRA,9559.0,144.02846,Brazil,100,1,2008,0.54,184006481000,4807.48
8401,BRA,9559.0,144.02846,Brazil,100,1,2008,0.54,188167356000,4807.48
8402,BRA,9559.0,144.02846,Brazil,100,1,2008,0.54,190130443000,4807.48
8403,BRA,9559.0,144.02846,Brazil,100,1,2008,0.54,192030362000,4807.48
8404,BRA,9559.0,144.02846,Brazil,100,1,2008,0.54,193886508000,4807.48
...,...,...,...,...,...,...,...,...,...,...
80588,ZAF,9602.0,82408.55000,South Africa,100,100,2008,0.63,49119759000,5617.90
80589,ZAF,9602.0,82408.55000,South Africa,100,100,2008,0.63,49779471000,5617.90
80590,ZAF,9602.0,82408.55000,South Africa,100,100,2008,0.63,50477011000,5617.90
80591,ZAF,9602.0,82408.55000,South Africa,100,100,2008,0.63,51216964000,5617.90


In [ ]:
plt.figure(figsize=(8, 6))
sns.lineplot(x='quantile', y='income', hue='country', data=incomesSelected)
plt.yscale('log')
plt.xlabel('Quantile')
plt.ylabel('Revenu moyen')
plt.title('Distribution de revenus par quantile pour les pays choisis')
plt.legend()
plt.show()

In [ ]:
incomeFrench = incomes.loc[incomes.country == 'FRA', ['income', 'quantile','country','nation']]
incomeFrench

In [ ]:
plt.plot(incomeFrench['quantile'], incomeFrench['income'])
plt.title('Distribution de revenus en France')
plt.xlabel('centiles')
plt.ylabel('revenus en $')

# Mission 3